In [26]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.examples.tutorials.mnist import input_data

rng = np.random.RandomState(1234)
random_state = 42

In [4]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
mnist_X, mnist_y = mnist.train.images, mnist.train.labels
mnist_X = mnist_X.reshape((mnist_X.shape[0], 28, 28, 1))

train_X, valid_X, train_y, valid_y = train_test_split(mnist_X, mnist_y, test_size=0.1, random_state=42)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [22]:
class BatchNorm:
    def __init__(self, shape, epsilon=np.float32(1e-5)):
        self.gamma = tf.Variable(np.ones(shape, dtype='float32'), name='gamma')
        self.beta  = tf.Variable(np.zeros(shape, dtype='float32'), name='beta')
        self.epsilon = epsilon

    def f_prop(self, x):
        if len(x.get_shape()) == 2:
            mean, var = tf.nn.moments(x, axes=0, keepdims=True)
            std = tf.sqrt(var + self.epsilon)
        elif len(x.get_shape()) == 4:
            mean, var = tf.nn.moments(x, axes=(0,1,2), keep_dims=True)
            std = tf.sqrt(var + self.epsilon)
        normalized_x = (x - mean) / std
        return self.gamma * normalized_x + self.beta
    
class Conv:
    def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
        #"""
        # Xavier Initialization
        fan_in = np.prod(filter_shape[:3]) #配列の最初の３つの積
        fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
        self.W = tf.Variable(rng.uniform(
                        low=-np.sqrt(6/(fan_in + fan_out)),
                        high=np.sqrt(6/(fan_in + fan_out)),
                        size=filter_shape
                    ).astype('float32'), name='W')
        """
        initial = tf.truncated_normal(filter_shape, stddev=0.1)
        self.W = tf.Variable(initial)
        """ 
        self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b') # バイアスはフィルタごと
        self.function = function
        self.strides = strides
        self.padding = padding

    def f_prop(self, x):
        u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding) + self.b
        return self.function(u)
    
class Pooling:
    def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID'):
        self.ksize = ksize
        self.strides = strides
        self.padding = padding
    
    def f_prop(self, x):
        return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
    
class Flatten:
    def f_prop(self, x):
        return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
    
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        # Xavier
        self.W = tf.Variable(rng.uniform(
                        low=-np.sqrt(6/(in_dim + out_dim)),
                        high=np.sqrt(6/(in_dim + out_dim)),
                        size=(in_dim, out_dim)
                    ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
        self.function = function

    def f_prop(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)
    
class Activation:
    def __init__(self, function=lambda x: x):
        self.function = function
    
    def f_prop(self, x):
        return self.function(x)

In [23]:
layers = [ # (縦の次元数)x(横の次元数)x(チャネル数)
    Conv((3, 3, 3, 32)), # 32x32x3 -> 30x30x32
    BatchNorm((30, 30, 32)),
    Activation(tf.nn.relu),
    Pooling((1, 2, 2, 1)), # 30x30x32 -> 15x15x32
    Conv((3, 3, 32, 64)), # 15x15x32 -> 13x13x64
    BatchNorm((13, 13, 64)),
    Activation(tf.nn.relu),
    Pooling(((1, 2, 2, 1))), # 13x13x64 -> 6x6x64
    Conv((3, 3, 64, 128)), # 6x6x64 -> 4x4x128
    BatchNorm((4, 4, 128)),
    Activation(tf.nn.relu),
    Pooling((1, 2, 2, 1)), # 4x4x128 -> 2x2x128
    Flatten(),
    Dense(2*2*128, 256, tf.nn.relu),
    Dense(256, 10, tf.nn.softmax)
]

layers = [                            # (縦の次元数)x(横の次元数)x(チャネル数)
    Conv((5, 5, 1, 20)),  # 28x28x 1 -> 24x24x20
    BatchNorm((24, 24, 20)),
    Activation(tf.nn.relu),
    Pooling((1, 2, 2, 1)),            # 24x24x20 -> 12x12x20
    Conv((5, 5, 20, 50)), # 12x12x20 ->  8x 8x50
    BatchNorm((8, 8, 50)),
    Activation(tf.nn.relu),
    Pooling((1, 2, 2, 1)),            #  8x 8x50 ->  4x 4x50
    Flatten(),
    Dense(4*4*50, 10, tf.nn.softmax)
]



x = tf.placeholder(tf.float32, [None, 28, 28, 1])
t = tf.placeholder(tf.float32, [None, 10])

def f_props(layers, x):
    for layer in layers:
        x = layer.f_prop(x)
    return x

y = f_props(layers, x)

cost = -tf.reduce_mean(tf.reduce_sum(t * tf.log(tf.clip_by_value(y, 1e-10, 1.0)), axis=1))
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

valid = tf.argmax(y, 1)

In [24]:
n_epochs = 1#10
batch_size = 100
n_batches = train_X.shape[0]//batch_size

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        train_X, train_y = shuffle(train_X, train_y, random_state=random_state)
        for i in range(n_batches):
            start = i * batch_size
            end = start + batch_size
            sess.run(train, feed_dict={x: train_X[start:end], t: train_y[start:end]})
        pred_y, valid_cost = sess.run([valid, cost], feed_dict={x: valid_X, t: valid_y})
        print('EPOCH:: %i, Validation cost: %.3f, Validation F1: %.3f' % (epoch + 1, valid_cost, f1_score(np.argmax(valid_y, 1).astype('int32'), pred_y, average='macro')))

EPOCH:: 1, Validation cost: 0.145, Validation F1: 0.959
